In [ ]:
import pandas as pd
from data_extraction.dummy_data_extractor import extract_dummy_data
from sklearn.metrics import mean_squared_error
from statsforecast import StatsForecast

import re
from statsforecast.models import (
    # HoltWinters,
    # CrostonClassic as Croston, 
    # HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive,
    # AutoARIMA
)

In [465]:
master_table = extract_dummy_data("dummy_data")

### Models

In [466]:
row = master_table.iloc[0]

df = row["his"]#.to_dataframe()                           #### IMPORTANT : UNCOMMENT THIS ON SS
df.set_index(df.columns[0], inplace=True, drop=True)
length_of_missing_data = row["dqDuration"]
data_logging_interval = row["pointInterval"]


In [467]:
def seasonal_naive(df, length_of_missing_data, data_logging_interval):
    """
    Inputs
    df: df used for training set (from SS)
    length_of_missing_data: interval length of missing data (from SS)
    data_logging_interval: data logging interval - called from the hisDQInterval tag on the point (from SS)

    Output
    forecasts_df: dataframe with predictions for the period missing data. Index names as ts, values column named as "v0
    """
    

    # step 1 convert the grid to a dataframe, and set first column as index     ### UNCOMMENT THIS ONLY IF RUNNING THE MODEL DIRECTLY ON SS. THIS IS DONE IN THE ENSEMBLE MODEL SO NO NEED TO HAVE THIS WHEN RUNNING THROUGH ENSEMBLE MODEL
    #df = df.to_dataframe()
    #df.set_index(df.columns[0], inplace=True, drop=True)

    # rename the first column as "target"
    new_column_name = "target"
    df = df.rename(columns={df.columns[0]: new_column_name})

    # format the df to statsforecast format
    df = df.reset_index()
    df = df.rename(columns={df.columns[0]: 'ds', df.columns[1]: "y"})
    df['unique_id'] = "v0"    

    # number of predictions
    horizon = int(length_of_missing_data/data_logging_interval) + 1 # why -1? because if you do length_of_missing_data/data_logging_interval you will get prediction length that is exclusive of the start ts (start ts is the last ts with actual data before the gap), and inclusive of the end ts (end ts is the first ts with actual data after the gap). +1 to get predictions also for the start and end timestamp. Can remove them later

    # season length
    season_length = int(pd.Timedelta(24, 'h') / data_logging_interval)      

    # frequency
    freq = str(data_logging_interval.total_seconds()/3600)+"h"


    # LIST OF MODELS
    models = [
        SeasonalNaive(season_length=season_length) 
    ]

    # The Model
    sf = StatsForecast( 
        models=models,
        freq=freq, 
        # fallback_model = SeasonalNaive(season_length=season_length),
        n_jobs=-1,
    )

    # Model fitting
    forecasts_df = sf.forecast(df=df[["ds", "y", "unique_id"]], h=horizon, level=[90])  

    # removing the -hi- and -lo- columns
    for col in forecasts_df.columns:
        if re.search("-hi-", col) or re.search("-lo-", col):
            forecasts_df.drop(col, axis=1, inplace=True)
            
    forecasts_df = forecasts_df.rename(columns={"ds": "timestamp", "SeasonalNaive":"seasonalNaive"})

    forecasts_df.set_index("timestamp", inplace=True)

    return forecasts_df

In [469]:
def dynamic_optimized_theta(df, length_of_missing_data, data_logging_interval):
    """
    Inputs
    df: df used for training set (from SS)
    length_of_missing_data: interval length of missing data (from SS)
    data_logging_interval: data logging interval - called from the hisDQInterval tag on the point (from SS)

    Output
    forecasts_df: dataframe with predictions for the period missing data. Index names as ts, values column named as "v0
    """
    

    # step 1 convert the grid to a dataframe, and set first column as index     ### UNCOMMENT THIS ONLY IF RUNNING THE MODEL DIRECTLY ON SS. THIS IS DONE IN THE ENSEMBLE MODEL SO NO NEED TO HAVE THIS WHEN RUNNING THROUGH ENSEMBLE MODEL
    #df = df.to_dataframe()
    #df.set_index(df.columns[0], inplace=True, drop=True)

    # rename the first column as "target"
    new_column_name = "target"
    df = df.rename(columns={df.columns[0]: new_column_name})

    # format the df to statsforecast format
    df = df.reset_index()
    df = df.rename(columns={df.columns[0]: 'ds', df.columns[1]: "y"})
    df['unique_id'] = "v0"    

    # number of predictions
    horizon = int(length_of_missing_data/data_logging_interval) + 1 # why -1? because if you do length_of_missing_data/data_logging_interval you will get prediction length that is exclusive of the start ts (start ts is the last ts with actual data before the gap), and inclusive of the end ts (end ts is the first ts with actual data after the gap). +1 to get predictions also for the start and end timestamp. Can remove them later

    # season length
    season_length = int(pd.Timedelta(24, 'h') / data_logging_interval)      

    # frequency
    freq = str(data_logging_interval.total_seconds()/3600)+"h"


    # LIST OF MODELS
    models = [
        DOT(season_length=season_length) 
    ]

    # The Model
    sf = StatsForecast( 
        models=models,
        freq=freq, 
        # fallback_model = SeasonalNaive(season_length=season_length),
        n_jobs=-1,
    )

    # Model fitting
    forecasts_df = sf.forecast(df=df[["ds", "y", "unique_id"]], h=horizon, level=[90])  

    # removing the -hi- and -lo- columns
    for col in forecasts_df.columns:
        if re.search("-hi-", col) or re.search("-lo-", col):
            forecasts_df.drop(col, axis=1, inplace=True)
            
    forecasts_df = forecasts_df.rename(columns={"ds": "timestamp", "DynamicOptimizedTheta":"dynamicOptimizedTheta"})

    forecasts_df.set_index("timestamp", inplace=True)

    return forecasts_df

In [452]:
def ensemble_model(python_master_table):
    """
    Function to run all models, and return the one with lowest RMSE.
    Models running through the ensemble model will have input DataFrame (AKA the "his" column on master_table) 
    with timestamp as index, target variable as first column, feature variables as the rest of the columns.

    Make sure the output predictions of all models are INCLUSIVE of both the "start ts" and "end ts" (AKA
    last ts with real data before gap, and first ts with real data after gap) 

    Make sure to follow camelCase for DataFrame column naming for compatibility with SS
    """

    # dictionary to save predictions for each point
    scores_df_dict = {
    "pointID": [],
    "predictions": [],
    "rmse": [],
    "modelName": []
    }

    # Create a DataFrame from the dictionary
    scores_df = pd.DataFrame(scores_df_dict)

    for i, row in python_master_table.iterrows():

        #-----------------
        # INPUTS TO MODELS
        #-----------------

        pointID = row["pointID"]
        df = row["his"]#.to_dataframe()                           #### IMPORTANT : UNCOMMENT THIS ON SS
        df.set_index(df.columns[0], inplace=True, drop=True)
        length_of_missing_data = row["dqDuration"]
        data_logging_interval = row["pointInterval"]


        #----------------------------
        # Dict of Data Quality Models                              ############# ADD NEW MODELS HERE 
        #----------------------------

        dq_models = {
            "Seasonal Naive" : seasonal_naive,
            "Dynamic Optimized Theta": dynamic_optimized_theta
        }

        for model_name, model in dq_models.items():
            
            #------------------------
            # ** Calculating RMSE **
            #------------------------

            # number of predictions needed
            horizon = int(length_of_missing_data/data_logging_interval) +1 # why +1? because if you do length_of_missing_data/data_logging_interval you will get prediction length that is exclusive of the start ts (start ts is the last ts with actual data before the gap), and inclusive of the end ts (end ts is the first ts with actual data after the gap). +1 to get predictions INCLUSIVE of BOTH start and end ts

            # training set size (relative to the horizon/prediction size)
            training_set_size = horizon * 10

            # training / testing set to evaluate the model (relative to horizon of prediction)
            train_data = df.iloc[-1*int(training_set_size):-1*int(horizon)]
            test_data = df.iloc[-1*int(horizon):]

            # the prediction. USED ONLY TO EVALUATE RMSE
            seasonal_naive_predictions_for_rmse = model(df = train_data, length_of_missing_data = length_of_missing_data, data_logging_interval = data_logging_interval)
            rmse_score = mean_squared_error(test_data[test_data.columns[0]].to_numpy(), seasonal_naive_predictions_for_rmse[seasonal_naive_predictions_for_rmse.columns[0]].to_numpy(), squared=False)

            #------------------
            # ** Predictions **
            #------------------

            # the predictions. USED FOR DATA CLEANING (uses all the data as training)
            seasonal_naive_predictions = model(df, length_of_missing_data, data_logging_interval)

            # keep only timestamps for null periods (rows where there are null values on SS)
            start = row['dqStart']
            duration = row['dqDuration']
            interval = row['pointInterval']
            timestamps = pd.date_range(start=start, end=start + duration, freq=interval)[1:-1] # clipping the first and last timestamps, as they already exist with actual data on SS

            seasonal_naive_predictions = seasonal_naive_predictions[seasonal_naive_predictions.index.isin(timestamps)]

            # reset index to make the ts a column instead of index. SS doesnt show the index of a DF
            seasonal_naive_predictions = seasonal_naive_predictions.reset_index()

            # append data to the scores DF
            row_to_append = {'pointID': pointID, 'predictions': seasonal_naive_predictions, 
                            "rmse": rmse_score, "modelName": model_name, 
                            "identifier": 
                                str(row["pointID"])
                                +str(row["dqStart"])
                                +str(row["dqDuration"])
                                +str(row["dqType"])}
            
            scores_df = pd.concat([scores_df, pd.DataFrame([row_to_append])], ignore_index=True)

            # return predictions with least RMSE for each point/dq issue
            idx = scores_df.groupby('identifier')['rmse'].idxmin()
            scores_df = scores_df.loc[idx].reset_index(drop=True)
            
    return scores_df    


In [453]:
a = ensemble_model(master_table)


c:\Users\abbayoumi\AppData\Local\anaconda3\envs\env_full\lib\site-packages\statsforecast\core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\abbayoumi\AppData\Local\anaconda3\envs\env_full\lib\site-packages\statsforecast\core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\abbayoumi\AppData\Local\anaconda3\envs\env_full\lib\site-packages\statsforecast\core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
c:\Users\abbayoumi\AppData\Local\anaconda3\envs\env_full\lib\sit

In [471]:

a#.at[1, "predictions"]

,pointID,predictions,rmse,modelName,identifier
0,@p:dmc_All:r:2ddf07d5-ef59ca94 DMC Building 1 ...,timestamp SeasonalNaive 0...,0.141386,Seasonal Naive,@p:dmc_All:r:2ddf07d5-ef59ca94 DMC Building 1 ...
1,@p:dmc_All:r:2ddf07d5-ef59ca94 DMC Building 1 ...,timestamp DynamicOptimize...,0.574149,Dynamic Optimized Theta,@p:dmc_All:r:2ddf07d5-ef59ca94 DMC Building 1 ...
